In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt

import time
import pickle

from datetime import datetime
from pathlib import Path

import math 

In [ ]:
%run "custom_datasets.ipynb"


In [ ]:
def set_device():
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return DEVICE 

DEVICE = set_device()
# DEVICE = torch.device('cpu')

print('----------------------------------')
print('Using device for training:', DEVICE)
print('----------------------------------')

In [ ]:
image_size = "full"

data = PrepareData3D(["Aorta Volunteers", "Aorta BaV", "Aorta Resvcue", "Aorta CoA"], 
                     image_size=image_size, norm_min_max=[0,1])

train_ds = SirenDataset(data.train, DEVICE) 
train_dataloader = DataLoader(train_ds, batch_size=1, num_workers=0, shuffle=True)
print(train_ds.__len__())
mask = next(iter(train_dataloader))[6].view(128, 128, 24)
mask = mask.detach().cpu().numpy()

print(mask.shape)

In [ ]:
import plotly.graph_objects as go
import numpy as np

X, Y, Z = np.mgrid[-8:8:24j, -8:8:128j, -8:8:128j]
# values = np.sin(X*Y*Z) / (X*Y*Z)
values = mask

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.1,
    isomax=1,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=15, # needs to be a large number for good volume rendering
    ))
fig.show()

In [ ]:
ax = plt.figure(figsize=(15, 15)).add_subplot(projection='3d')
ax.voxels(mask)

plt.show()